In [1]:
# -*- coding: utf-8 -*- 
import sys
import random
#reload(sys)
#sys.setdefaultencoding("utf-8")
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import logging
import os
import io
import glob
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import gc
pd.set_option('display.min_rows',None)
from sklearn.metrics import roc_curve,auc,roc_auc_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import datetime, time
import traceback
from gensim.models import Word2Vec
from pandarallel import pandarallel
pandarallel.initialize()
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from scipy import interpolate
import implicit
from scipy.sparse import coo_matrix
color = sns.color_palette()
import sys  
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
#from func_utils_v14_sample_no_reduce_mem import *
import math
import multiprocessing
#import treelite
#import treelite_runtime     # runtime module
import polars as pl
import pyarrow.parquet as pq
random.seed(2027) # n就是你想设置的随机种子
np.random.seed(1997) # n就是你想设置的随机种子
#reload(sys)
#sys.setdefaultencoding('gbk') 
mpl.rcParams['font.sans-serif'] = ['SimHei']
%matplotlib inline

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
def drop_unuse_cols_by_pandas (df, unuse_cols):
    for col in unuse_cols :
        if col in df.columns.values :
            del df[col]
    return df    

def drop_unuse_cols_by_polars (df, unuse_cols):
    for col in unuse_cols :
        if col in df.columns :
            df = df.drop(col)
    return df  

def select_part_df_by_id(df, name, val_set) :
    return df[df[name].isin(val_set)].reset_index(drop=True)


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    cols_ = [col for col in list(df) if col not in ['cid', 'vid']]
    for col in cols_:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def reduce_memory_usage_pl(df, name):
    """ Reduce memory usage by polars dataframe {df} with name {name} by changing its data types.
        Original pandas version of this function: https://www.kaggle.com/code/arjanso/reducing-dataframe-memory-size-by-65 """
    print(f"Memory usage of dataframe {name} is {round(df.estimated_size('mb'), 2)} MB")
    Numeric_Int_types = [pl.Int8,pl.Int16,pl.Int32,pl.Int64]
    Numeric_Float_types = [pl.Float32,pl.Float64]    
    Cate_types = [pl.Categorical, pl.Utf8]
    #for col in df.columns:
    for col in [col for col in df.columns if col != 'did'] :
        col_type = df[col].dtype
        if col_type not in Numeric_Int_types + Numeric_Float_types + Cate_types :
            continue
        c_min = df[col].min()
        c_max = df[col].max()
        if col_type in Numeric_Int_types:
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df = df.with_columns(df[col].cast(pl.Int8))
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df = df.with_columns(df[col].cast(pl.Int16))
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df = df.with_columns(df[col].cast(pl.Int32))
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                df = df.with_columns(df[col].cast(pl.Int64))
        elif col_type in Numeric_Float_types:
            try :
                #if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                #    df = df.with_columns(df[col].cast(pl.Float16))
                #elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                #    df = df.with_columns(df[col].cast(pl.Float32))
                #else:
                    #df[col] = df[col].astype(np.float64)                   
                #    df = df.with_columns(df[col].cast(pl.Float64))
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df = df.with_columns(df[col].cast(pl.Float32))
                else:
                    df = df.with_columns(df[col].cast(pl.Float32))
                    #pass
            except :
                print ("reduce memory exception: ", name, col, c_min, c_max)
                pass
        elif col_type == pl.Utf8 :
            df = df.with_columns(df[col].cast(pl.Categorical))
        else:
            pass
    print(f"Memory usage of dataframe {name} became {round(df.estimated_size('mb'), 2)} MB")
    return df

In [3]:
def agg_expr(agg, group_name):
    expr_max = [pl.max(col).alias(f"max_{group_name}_{col}") for col, methods in agg.items() for method in methods if method.lower() == 'max']
    expr_mean = [pl.mean(col).alias(f"mean_{group_name}_{col}") for col, methods in agg.items() for method in methods if method.lower() == 'mean']
    expr_min = [pl.min(col).alias(f"min_{group_name}_{col}") for col, methods in agg.items() for method in methods if method.lower() == 'min']
    expr_std = [pl.std(col).alias(f"std_{group_name}_{col}") for col, methods in agg.items() for method in methods if method.lower() == 'std']
    expr_count = [pl.count(col).alias(f"count_{group_name}_{col}") for col, methods in agg.items() for method in methods if method.lower() == 'count']
    expr_sum = [pl.sum(col).alias(f"sum_{group_name}_{col}") for col, methods in agg.items() for method in methods if method.lower() == 'sum']
    return expr_max + expr_mean + expr_std + expr_min + expr_count + expr_sum

def get_day (df, col) :
    df = df.with_columns(
        pl.col(col).map_elements(lambda x : x.split(' ')[0]).alias('day')
    )
    return df

def make_vid_pair_sim (df, group_col, use_cols, agg) :
    df_left = df.select(group_col + use_cols)#.rename({'vid' : 'left_vid', 'play_time' : 'left_play_time'})
    df_right = df.select(group_col + use_cols)#.rename({'vid' : 'right_vid', 'play_time' : 'right_play_time'})
    df_left = df_left.join(df_right, on=group_col, how='left')
    #df_left = df_left.filter(
    #    pl.col('vid') != pl.col('vid_right')
    #)
    df_features = df_left.groupby(['vid', 'vid_right'], maintain_order=True).agg(agg_expr(agg, 'vid_pair_sim'))
    return df_features

#召回构建训练样本
def create_data_sample (df_history_short_click_playtime, 
                        df_label,
                        history_seq_num,
                        df_candidate_did, 
                        df_candidate_vid, 
                        df_pair_vid_sim,
                        top_pair_sim_N=50, 
                        hots_N=10,
                        stage='train') :
    df_sample = df_history_short_click_playtime.filter(
        pl.col('did').is_in(df_candidate_did['did'].unique().to_list())
    ).groupby('did').tail(history_seq_num)
    df_sample = df_sample.join(df_pair_vid_sim.groupby('vid').head(top_pair_sim_N), on='vid', how='left')
    df_sample = df_sample.drop('vid')
    df_sample = df_sample.rename({'vid_right' : 'vid'})
    df_sample = df_sample.select(['did', 'vid', 'count_vid_pair_sim_did', 'sum_vid_pair_sim_play_time_right'])

    #每个did补label日最近一天的热门vid
    hots_N_vids = df_history_short_click_playtime.filter(
        pl.col('day') == df_history_short_click_playtime['day'].max()
    )['vid'].value_counts().sort('count', descending=True)['vid'].to_list()[0:hots_N]
    df_added_sample = df_candidate_did.with_columns(
        pl.lit(hots_N_vids).alias('hots_N_vids')
    )
    #补热门
    df_added_sample = df_added_sample.explode('hots_N_vids').rename({'hots_N_vids' : 'vid'})
    df_sample = pl.concat([df_sample, df_added_sample], how="diagonal").unique(('did', 'vid'), keep='first', maintain_order=True)
    df_sample = df_sample.fill_null(0)
    
    #只保留候选vid的样本
    df_sample = df_sample.filter (
        pl.col('vid').is_in(df_candidate_vid['vid'].to_list())
    )
    #测试集不必构造label
    if stage != 'test' :
        df_label = df_label.select(['did', 'vid']).unique()
        df_label = df_label.with_columns(
            pl.lit(1).alias('label')
        )
        df_sample = df_sample.join(df_label, on=['did', 'vid'], how='left')

        #构建正负样本
        df_sample = df_sample.with_columns(
            pl.col('label').fill_null(0)
        )    
    return df_sample

#from datetime import datetime
def str_to_date (dt_str) :
    dt_obj = datetime.datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S')
    return dt_obj

def make_features (df_sample, df_history_short_click_playtime, df_history_short_show, vid_info):
    agg = {
        'vid' : ['count'], 
        'play_time' : ['mean', 'std', 'max', 'min'],
    }
    df_did_click_features = df_history_short_click_playtime.groupby('did').agg(agg_expr(agg, 'did_history_click'))
    agg = {
        'did' : ['count'],
        'play_time' : ['mean', 'std', 'max', 'min'],
    }
    df_vid_click_features = df_history_short_click_playtime.groupby('vid').agg(agg_expr(agg, 'vid_history_click'))
    
    agg = {
        'did' : ['count'],
        'play_time' : ['mean', 'std', 'max', 'min'],
    }
    df_cid_click_features = df_history_short_click_playtime.groupby('cid').agg(agg_expr(agg, 'cid_history_click'))    
    
    agg = {
        'play_time' : ['count'],
    }
    df_did_vid_click_features = df_history_short_click_playtime.groupby(['did', 'vid']).agg(agg_expr(agg, 'did_vid_history_click'))
    
    agg = {
        'play_time' : ['count', 'mean', 'std', 'max', 'min'],
    }
    df_did_cid_click_features = df_history_short_click_playtime.groupby(['did', 'cid']).agg(agg_expr(agg, 'did_cid_history_click'))
    
    agg = {
        'vid' : ['count'], 
    }
    df_did_show_features = df_history_short_show.groupby('did').agg(agg_expr(agg, 'did_history_show'))
    
    agg = {
        'did' : ['count'], 
    }
    df_vid_show_features = df_history_short_show.groupby('vid').agg(agg_expr(agg, 'vid_history_show'))

    agg = {
        'did' : ['count'], 
    }
    df_cid_show_features = df_history_short_show.groupby('cid').agg(agg_expr(agg, 'cid_history_show'))
    
    agg = {
        'show_time' : ['count'],
    }
    df_did_vid_show_features = df_history_short_show.groupby(['did', 'vid']).agg(agg_expr(agg, 'did_vid_history_show'))
    
    agg = {
        'show_time' : ['count'],
    }
    df_did_cid_show_features = df_history_short_show.groupby(['did', 'cid']).agg(agg_expr(agg, 'did_cid_history_show'))    
    
    df_sample = df_sample.join(vid_info.select(['vid', 'cid', 'is_intact', 'classify_id', 'serialno', 'duration', 'online_time']), 
                               on='vid', 
                               how='left')
    df_sample = df_sample.with_columns(
        (df_history_short_show['show_time'].max() - pl.col('online_time')).alias('online_time_gap') / (1_000_000 * 60 * 60 * 24)
    )    
    df_sample = df_sample.join(df_did_click_features, on='did', how='left')
    df_sample = df_sample.join(df_vid_click_features, on='vid', how='left')
    df_sample = df_sample.join(df_cid_click_features, on='cid', how='left')
    df_sample = df_sample.join(df_did_vid_click_features, on=['did', 'vid'], how='left')
    df_sample = df_sample.join(df_did_cid_click_features, on=['did', 'cid'], how='left')
    df_sample = df_sample.join(df_did_show_features, on='did', how='left')
    df_sample = df_sample.join(df_vid_show_features, on='vid', how='left')
    df_sample = df_sample.join(df_cid_show_features, on='cid', how='left')
    df_sample = df_sample.join(df_did_vid_show_features, on=['did', 'vid'], how='left')
    df_sample = df_sample.join(df_did_cid_show_features, on=['did', 'cid'], how='left')

    df_sample = df_sample.with_columns(
        (pl.col('count_vid_history_click_did') / (pl.col('count_vid_history_show_did') + 0.01)).alias('vid_ctr'),
        (pl.col('count_cid_history_click_did') / (pl.col('count_cid_history_show_did') + 0.01)).alias('cid_ctr'),
    )
    return df_sample

def make_pipline(df_history_short_click_playtime, df_history_short_show, df_label, df_candidate_did, df_candidate_vid, stage) :
    agg = {
        'did' : ['count'], 
        'play_time_right' : ['sum'],
    }
    df_pair_click_sim = make_vid_pair_sim (df_history_short_click_playtime, 
                                           group_col=['did'], 
                                           use_cols=['vid', 'play_time'],
                                           agg=agg) 
    df_pair_click_sim = df_pair_click_sim.sort(['vid', 'count_vid_pair_sim_did'], descending=True)

    df_sample = create_data_sample (df_history_short_click_playtime,
                                          df_label,
                                          history_seq_num=10,
                                          df_candidate_did=df_candidate_did, 
                                          df_candidate_vid=df_candidate_vid, 
                                          df_pair_vid_sim=df_pair_click_sim,
                                          top_pair_sim_N=50, 
                                          hots_N=20,
                                          stage=stage)

    df_sample = make_features (df_sample, df_history_short_click_playtime, 
                                     df_history_short_show, vid_info)
    return df_sample

In [4]:
def RR(ranked_list, ground_list):
    for i in range(len(ranked_list)):
        id = ranked_list[i]
        if id in ground_list:
            return 1 / (i + 1.0)
    return 0

def merge_answser (df_results, df_answer) :
    #df_results = df_results.sort_values(by=['did', 'rank']).reset_index(drop=True)
    df_results = df_results.groupby('did')['vid'].apply(list).reset_index()
    df_results.rename(columns = {'vid' : 'vid_list'}, inplace=True)
    #df_results = df_results.merge(df_answer[['did', 'vid']], on='did', how='left')
    #df_results['vid'] = df_results['vid'].apply(lambda x : [x])
    df_results['vid'] = df_results['did'].map(df_answer.groupby('did')['vid'].apply(list))
    return df_results

def get_vid_sim (vid1, vid2) :
    if vid1 in vid_to_tags and vid2 in vid_to_tags :
        return len(set(vid_to_tags[vid1]) & set(vid_to_tags[vid2])) / len(set(vid_to_tags[vid1]) | set(vid_to_tags[vid2])) 
    return 1


def cal_mrr_score (df_results, df_answer) :
    df_results = merge_answser (df_results, df_answer)
    #df_results['base_model_v1_rr_score'] = df_results[['vid', 'vid_list']].parallel_apply(lambda x : RR(x['vid'], x['vid_list']), axis=1)
    df_results['base_model_v1_rr_score'] = df_results[['vid_list', 'vid']].apply(lambda x : RR(x['vid_list'], x['vid']), axis=1)
    base_model_v1_rr_score = df_results['base_model_v1_rr_score'].mean()
    print ("base_model_v1_rr_score: ", base_model_v1_rr_score)    
    return base_model_v1_rr_score

def compute_diversity_sim(elements):
    """
    Compute pairwise absolute differences for a list of elements without repetition.
    """
    similarities = {}
    sim_sum = 0
    for i in range(len(elements)):
        for j in range(i + 1, len(elements)):
            #sim = abs(elements[i] - elements[j])
            sim = get_vid_sim (elements[i], elements[j])
            similarities[(i, j)] = 1 - sim
            sim_sum += 1 - sim
    sim_sum *= 1 / (len(elements) * (len(elements) - 1))
    #return similarities, sim_sum
    return similarities, sim_sum

def cal_final_score (df_results, df_answer, weight=0.9) :
    df_temps = merge_answser (df_results, df_answer)
    #计算F1_Score
    df_temps['base_model_v1_rr_score'] = df_temps[['vid_list', 'vid']].apply(lambda x : RR(x['vid_list'], x['vid']), axis=1)
    #计算多样性
    df_temps['diversity_score'] = df_temps['vid_list'].apply(lambda x : compute_diversity_sim(x)[1])
    df_temps['final_score'] = df_temps[['base_model_v1_rr_score', 'diversity_score']].apply(lambda x : x['base_model_v1_rr_score'] * weight + x['diversity_score'] * (1-weight) if x['base_model_v1_rr_score'] > 0 else x['base_model_v1_rr_score'], axis=1)    
    base_model_v1_rr_score = df_temps['base_model_v1_rr_score'].mean()
    diversity_score = df_temps['diversity_score'].mean()
    final_score = df_temps['final_score'].mean()
    print ("base_model_v1_rr_score:%s", base_model_v1_rr_score)    
    print ("diversity_score:%s", diversity_score)
    print ("final_score:%s", final_score)
    return df_temps, final_score

In [5]:
%%time
flag = 1
df_short_click_playtime_set = []
df_short_show_set = []
for day in tqdm(['day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7', 'day_8', 'day_9', 'day_10', 'day_11', 'day_12',]) :
    df_short_click_playtime = pl.read_csv(f'data_v4/A/{day}/short_click_playtime.csv')  
    df_short_click_playtime = df_short_click_playtime.with_columns (
        pl.lit(flag).alias('day')
    )
    df_short_click_playtime_set.append(df_short_click_playtime)
    
    df_short_show = pl.read_csv(f'data_v4/A/{day}/short_show.csv')  
    df_short_show = df_short_show.with_columns (
        pl.lit(flag).alias('day')
    )
    df_short_show_set.append(df_short_show)    
    flag += 1
#df_short_click_playtime = reduce_memory_usage_pl(pl.concat(df_short_click_playtime_set), name='df_short_click_playtime_set')
df_short_click_playtime = pl.concat(df_short_click_playtime_set)
df_short_click_playtime = df_short_click_playtime.sort(['did', 'click_time'])

#df_short_show = reduce_memory_usage_pl(pl.concat(df_short_show_set), name='df_short_click_playtime_set')
df_short_show = pl.concat(df_short_show_set)
df_short_show = df_short_show.sort(['did', 'show_time'])

df_short_show = df_short_show.with_columns(pl.col("show_time").str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S'))
df_short_click_playtime = df_short_click_playtime.with_columns(pl.col("click_time").str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S'))

df_candidate_did_A = pl.read_csv('data_v4/A/df_candidate_did_A.csv')
df_candidate_vid_A = pl.read_csv('data_v4/A/df_candidate_vid_A.csv')

vid_info = pl.read_csv('data_v4/A/vid_info.csv')
vid_info = vid_info.with_columns(pl.col("online_time").str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S'))

df_short_show = df_short_show.join(vid_info.select(['vid', 'cid']), on='vid', how='left')
df_short_click_playtime = df_short_click_playtime.join(vid_info.select(['vid', 'cid']), on='vid', how='left')

#global vid_to_tags
df_tags = vid_info.select(['vid', 'stars']).to_pandas()
df_tags = df_tags[df_tags['stars'].notnull()].reset_index(drop=True)
#vid_to_tags = df_tags[['vid', 'stars']].set_index('vid')['stars'].apply(lambda x : x.split('|')).to_dict()
global vid_to_tags
vid_to_tags = df_tags[['vid', 'stars']].set_index('vid')['stars'].apply(lambda x : x.replace('[', '').replace(']', '').replace(' ', '').split(',')).apply(lambda x : [int(i) for i in x]).to_dict()

100%|██████████| 12/12 [00:13<00:00,  1.13s/it]


CPU times: user 13min 5s, sys: 51.1 s, total: 13min 56s
Wall time: 2min 41s


In [6]:
df_short_show

show_time,did,session_id,vid,day,cid
datetime[μs],str,str,i64,i32,i64
2024-01-07 22:11:47,"""0000008256d53103ddffc0fd1a2fc8…","""7401a46da969543e8c61a3b0244c7e…",1832750,7,15984
2024-01-07 22:11:47,"""0000008256d53103ddffc0fd1a2fc8…","""7401a46da969543e8c61a3b0244c7e…",2821241,7,22192
2024-01-07 22:11:47,"""0000008256d53103ddffc0fd1a2fc8…","""7401a46da969543e8c61a3b0244c7e…",2537858,7,17281
2024-01-06 19:52:08,"""000000de1fe68d8bb85f239b39b737…","""108f667c4fe70795e73502db2e4646…",1832750,6,15984
2024-01-06 19:52:08,"""000000de1fe68d8bb85f239b39b737…","""108f667c4fe70795e73502db2e4646…",27217,6,6195
…,…,…,…,…,…
2024-01-07 11:07:36,"""fffffd4f2145a17f308ca6e27906e2…","""de6c1ef52567c1ae43baead4b17020…",677286,7,3076
2024-01-07 11:07:36,"""fffffd4f2145a17f308ca6e27906e2…","""de6c1ef52567c1ae43baead4b17020…",976994,7,22690
2024-01-07 11:07:37,"""fffffd4f2145a17f308ca6e27906e2…","""de6c1ef52567c1ae43baead4b17020…",2970706,7,5207


In [7]:
df_short_click_playtime

did,click_time,play_time,session_id,vid,day,cid
str,datetime[μs],i64,str,i64,i32,i64
"""000008a67f875ee3bcd4dd29045fb0…",2024-01-11 09:34:52,125,"""b964b8735eecaffb1544fac65a3cce…",1246240,11,1940
"""000008a67f875ee3bcd4dd29045fb0…",2024-01-11 09:38:05,496,"""b964b8735eecaffb1544fac65a3cce…",459114,11,22928
"""00000e635d9e2773a18302e7e7df92…",2024-01-07 15:46:14,3252,"""843ff72ef5953f9cbe8b03aac4bdf6…",1914361,7,8701
"""00003236ac40bd7018e46aba68464a…",2024-01-12 21:10:13,123,"""bd303cd42f55fb2755f2d630449afa…",27217,12,6195
"""0000347998285d5b8c5a2da70e4c8a…",2024-01-04 13:28:18,437,"""7be17c9e6a43bbeb5ccc1dfa0cef5e…",1517901,4,22804
…,…,…,…,…,…,…
"""ffffbc9da1e97c38084fb1e8222155…",2024-01-07 18:12:39,5910,"""eef63bd6bbf20c11f13a33d445c98c…",2760684,7,11492
"""ffffbc9da1e97c38084fb1e8222155…",2024-01-08 00:18:21,6562,"""eef63bd6bbf20c11f13a33d445c98c…",2693554,8,6483
"""ffffe54ed94b225c439e76fdb762b8…",2024-01-11 20:51:17,19,"""7ccf7b9c9b3178ca139dd118a66548…",2804691,11,13210


In [8]:
%%time
#训练集提取day_11之前的特征，以day_11作为label
train_day = 11
stage = 'train'
df_train_label = df_short_click_playtime.filter(pl.col('day') == train_day)
df_train_history_short_click_playtime = df_short_click_playtime.filter(pl.col('day') < train_day)
df_train_history_short_show = df_short_show.filter(pl.col('day') < train_day)
df_candidate_did_train = df_train_label.select('did').unique()
df_candidate_vid_train = df_train_label.select('vid').unique()
df_train_sample = make_pipline(df_train_history_short_click_playtime, 
                               df_train_history_short_show, 
                               df_train_label, 
                               df_candidate_did_train, 
                               df_candidate_vid_train, stage=stage) 
df_train_sample = reduce_mem_usage(df_train_sample.to_pandas(), verbose=True)

#验证集提取day_12之前的特征，以day_12作为label
valid_day = 12
stage = 'valid'
df_valid_label = df_short_click_playtime.filter(pl.col('day') == valid_day)
df_valid_history_short_click_playtime = df_short_click_playtime.filter(pl.col('day') < valid_day)
df_valid_history_short_show = df_short_show.filter(pl.col('day') < valid_day)
df_candidate_did_valid = df_valid_label.select('did').unique()
df_candidate_vid_valid = df_valid_label.select('vid').unique()
df_valid_sample = make_pipline(df_valid_history_short_click_playtime, 
                               df_valid_history_short_show, 
                               df_valid_label, 
                               df_candidate_did_valid, 
                               df_candidate_vid_valid, stage=stage) 
#df_valid_sample = reduce_memory_usage_pl(df_valid_sample, name='df_valid_sample')
df_valid_sample = reduce_mem_usage(df_valid_sample.to_pandas(), verbose=True)

#测试集无label
#valid_day = 13
stage = 'test'
df_test_label = pl.DataFrame()
df_test_sample = make_pipline(df_short_click_playtime, 
                               df_short_show, 
                               df_test_label, 
                               df_candidate_did_A, 
                               df_candidate_vid_A, stage=stage) 
#df_valid_sample = reduce_memory_usage_pl(df_valid_sample, name='df_valid_sample')
df_test_sample = reduce_mem_usage(df_test_sample.to_pandas(), verbose=True)

gc.collect()

Mem. usage decreased to 871.06 Mb (56.3% reduction)
Mem. usage decreased to 934.82 Mb (55.7% reduction)
Mem. usage decreased to 998.90 Mb (56.1% reduction)
CPU times: user 25min 1s, sys: 8min 38s, total: 33min 40s
Wall time: 4min 57s


0

In [9]:
df_train_sample

,did,vid,count_vid_pair_sim_did,sum_vid_pair_sim_play_time_right,label,cid,is_intact,classify_id,serialno,duration,...,std_did_cid_history_click_play_time,min_did_cid_history_click_play_time,count_did_cid_history_click_play_time,count_did_history_show_vid,count_vid_history_show_did,count_cid_history_show_did,count_did_vid_history_show_show_time,count_did_cid_history_show_show_time,vid_ctr,cid_ctr
0,0d2fb24b6ac24bc69cf565a001e75788,6781,20824,29806576,0,21893,1,1,32,5332,...,NaN,1447.0,1.0,62.0,358585,2477865,1.0,3.0,0.049316,0.029099
1,0d2fb24b6ac24bc69cf565a001e75788,2804691,783,604891,0,13210,2,1,34,51,...,NaN,NaN,NaN,62.0,872405,4071695,NaN,NaN,0.100525,0.040649
2,0d2fb24b6ac24bc69cf565a001e75788,1386009,709,115972,0,9935,1,1,3,387,...,NaN,NaN,NaN,62.0,129808,141233,1.0,1.0,0.086365,0.081543
3,0d2fb24b6ac24bc69cf565a001e75788,3033280,641,853947,0,21893,1,1,19,5118,...,NaN,1447.0,1.0,62.0,587388,2477865,1.0,3.0,0.032410,0.029099
4,0d2fb24b6ac24bc69cf565a001e75788,945713,595,824853,0,12594,1,1,1,2685,...,NaN,NaN,NaN,62.0,140245,267135,NaN,NaN,0.075562,0.057068
5,0d2fb24b6ac24bc69cf565a001e75788,505996,589,1153497,0,14708,1,1,1,2938,...,NaN,NaN,NaN,62.0,157551,419496,1.0,1.0,0.068665,0.042755
6,0d2fb24b6ac24bc69cf565a001e75788,1471624,570,881390,0,15063,1,1,36,5145,...,NaN,NaN,NaN,62.0,184548,310698,1.0,1.0,0.032440,0.024399
7,0d2fb24b6ac24bc69cf565a001e75788,2386259,472,417811,0,17077,1,1,34,1158,...,NaN,NaN,NaN,62.0,384613,1227574,NaN,NaN,0.065247,0.030960
8,0d2fb24b6ac24bc69cf565a001e75788,3266012,385,503294,0,1268,1,1,4,6053,...,NaN,NaN,NaN,62.0,113727,474276,NaN,NaN,0.036926,0.024368
9,0d2fb24b6ac24bc69cf565a001e75788,2693554,369,294877,0,6483,1,1,32,6782,...,NaN,NaN,NaN,62.0,375879,1238959,NaN,NaN,0.059296,0.033569


In [10]:
df_valid_sample

,did,vid,count_vid_pair_sim_did,sum_vid_pair_sim_play_time_right,label,cid,is_intact,classify_id,serialno,duration,...,std_did_cid_history_click_play_time,min_did_cid_history_click_play_time,count_did_cid_history_click_play_time,count_did_history_show_vid,count_vid_history_show_did,count_cid_history_show_did,count_did_vid_history_show_show_time,count_did_cid_history_show_show_time,vid_ctr,cid_ctr
0,46bf13d5bfc72102a605b4b19baa58df,2905755,251,118014,0,25351,1,1,1,4437,...,NaN,1.0,1.0,278.0,21461,36237,1.0,3.0,0.010300,0.014214
1,46bf13d5bfc72102a605b4b19baa58df,2825604,17,5868,0,3698,1,1,28,7658,...,NaN,NaN,NaN,278.0,247554,578251,1.0,2.0,0.031235,0.024307
2,46bf13d5bfc72102a605b4b19baa58df,308090,14,276,0,9048,2,1,34,12,...,NaN,NaN,NaN,278.0,643167,1150872,NaN,2.0,0.019333,0.017517
3,46bf13d5bfc72102a605b4b19baa58df,1437231,6,4796,0,9048,1,1,38,2591,...,NaN,NaN,NaN,278.0,89761,1150872,NaN,2.0,0.013557,0.017517
4,46bf13d5bfc72102a605b4b19baa58df,1375783,6,237,0,12449,1,1,4,2196,...,NaN,NaN,NaN,278.0,57236,372281,1.0,2.0,0.026627,0.029358
5,46bf13d5bfc72102a605b4b19baa58df,1502893,5,66,0,17031,1,1,4,1508,...,NaN,NaN,NaN,278.0,32933,76001,1.0,2.0,0.020615,0.020935
6,46bf13d5bfc72102a605b4b19baa58df,3461449,5,519,0,13210,2,1,34,53,...,72.125,4.0,2.0,278.0,572277,4573772,1.0,29.0,0.020264,0.040619
7,46bf13d5bfc72102a605b4b19baa58df,668727,5,422,0,22449,2,1,1,123,...,NaN,NaN,NaN,278.0,90965,1756044,NaN,4.0,0.021027,0.015511
8,46bf13d5bfc72102a605b4b19baa58df,2804691,5,615,0,13210,2,1,34,51,...,72.125,4.0,2.0,278.0,1020560,4573772,1.0,29.0,0.097412,0.040619
9,46bf13d5bfc72102a605b4b19baa58df,2262873,5,18862,0,9048,2,1,25,174,...,NaN,NaN,NaN,278.0,84788,1150872,NaN,2.0,0.015144,0.017517


In [11]:
df_test_sample

,did,vid,count_vid_pair_sim_did,sum_vid_pair_sim_play_time_right,cid,is_intact,classify_id,serialno,duration,online_time,...,std_did_cid_history_click_play_time,min_did_cid_history_click_play_time,count_did_cid_history_click_play_time,count_did_history_show_vid,count_vid_history_show_did,count_cid_history_show_did,count_did_vid_history_show_show_time,count_did_cid_history_show_show_time,vid_ctr,cid_ctr
0,dc377d4ce0d35780633541bb683b3b1b,511037,14579,14056594,8586,1,1,13,9474,2023-02-08 11:55:13,...,NaN,44.0,1.0,24.0,214551,877226,1.0,1.0,0.058228,0.031494
1,dc377d4ce0d35780633541bb683b3b1b,2386259,943,1068915,17077,1,1,34,1158,2022-05-04 11:54:31,...,NaN,NaN,NaN,24.0,439915,1433867,1.0,1.0,0.063904,0.030121
2,dc377d4ce0d35780633541bb683b3b1b,2804691,571,248492,13210,2,1,34,51,2024-01-05 20:34:55,...,NaN,NaN,NaN,24.0,1194426,5141277,1.0,1.0,0.092773,0.040009
3,dc377d4ce0d35780633541bb683b3b1b,2693554,383,390453,6483,1,1,32,6782,2023-02-17 23:59:27,...,NaN,NaN,NaN,24.0,445668,1471212,1.0,1.0,0.058228,0.032990
4,dc377d4ce0d35780633541bb683b3b1b,71829,371,281843,20614,1,1,4,1376,2022-08-31 11:50:02,...,NaN,NaN,NaN,24.0,175796,1093487,NaN,1.0,0.057098,0.028046
5,dc377d4ce0d35780633541bb683b3b1b,3356103,289,512565,11492,2,1,20,86,2023-11-04 20:27:55,...,NaN,NaN,NaN,24.0,356834,1358740,NaN,1.0,0.062164,0.042511
6,dc377d4ce0d35780633541bb683b3b1b,2565164,242,50995,6,1,1,2,693,2022-01-06 11:56:45,...,NaN,NaN,NaN,24.0,182118,219872,NaN,NaN,0.040497,0.038513
7,dc377d4ce0d35780633541bb683b3b1b,6781,218,283595,21893,1,1,32,5332,2022-03-26 19:31:55,...,NaN,NaN,NaN,24.0,453057,3000555,NaN,1.0,0.047424,0.029297
8,dc377d4ce0d35780633541bb683b3b1b,1852,217,229180,8586,1,1,4,6293,2023-01-18 11:57:21,...,NaN,44.0,1.0,24.0,87318,877226,NaN,1.0,0.044495,0.031494
9,dc377d4ce0d35780633541bb683b3b1b,9084,212,470228,20614,1,1,15,8275,2022-09-22 11:55:24,...,NaN,NaN,NaN,24.0,123588,1093487,NaN,1.0,0.045197,0.028046


In [12]:
%%time
#训练模型
num = os.cpu_count()
print ("cpu_counts: ", num)

lgb_params = {
    #'objective': 'lambdarank',
    #'metric': {'ndcg'},
    #'ndcg_at': [10],
    #'group_column': 'did',      
    'objective': 'binary', #定义的目标函数
    #'metric': {'auc', 'binary_logloss'},
    'metric': {'auc'},
    'boosting_type' : 'gbdt',

    'learning_rate': 0.05,
    'max_depth' : 12,
    'num_leaves' : 2 ** 6,

    'min_child_weight' : 10,
    'min_data_in_leaf' : 40,

    'feature_fraction' : 0.70,
    'subsample' : 0.75,
    'seed' : 114,

    'nthread' : -1,
    'bagging_freq' : 1,
    'verbose' : -1,
    #'scale_pos_weight':200
}

train_label = df_train_sample['label'].values
valid_label = df_valid_sample['label'].values

df_train_group = df_train_sample.groupby("did")["did"].count().to_numpy()
df_valid_group = df_valid_sample.groupby("did")["did"].count().to_numpy()

unuse_cols_importance = ['did', 'label', 'online_time', 'predict_prob']

feature_name = df_train_sample.columns.values
feature_name = [i for i in feature_name if i not in unuse_cols_importance]
#feature_name = [col for col in feature_name if col not in unuse_cols_importance]
print(f"Feature length = {len(feature_name)}")
print ("Feature: ", feature_name)

print("Model Trainning.")

trn_data = lgb.Dataset(df_train_sample[feature_name], 
                       label=train_label,)
                       #group=df_train_group)#, categorical_feature=cat_cols)
val_data = lgb.Dataset(df_valid_sample[feature_name], 
                       label=valid_label,)
                       #group=df_valid_group)#, categorical_feature=cat_cols)

#print ("train_data : ", train_data.info())
#print ("test_data : ", train_data.info())
lgb_model = lgb.train(lgb_params,
                trn_data,
                1000,
                valid_sets=[val_data],
                #verbose_eval=50,
                #categorical_feature=cat_cols, 
                callbacks=[lgb.early_stopping(stopping_rounds=100), 
                           lgb.log_evaluation(period=100, show_stdv=True)])#, feval=self_gauc)        

df_valid_sample['predict_prob'] = lgb_model.predict(df_valid_sample[feature_name], 
                                                    num_iteration=lgb_model.best_iteration)


import time
path = "baseline_local_model_v1/" + str(int(time.time()))
print(time.time())#获当前时间的时间戳
print(path)#获取本地时间
os.makedirs(path)
lgb_model.save_model(path + '/model.txt')

#评估线下验证集得分
df_valid_result = df_valid_sample[['did', 'vid', 'predict_prob']].sort_values(by=['did', 'predict_prob'], ascending=False).reset_index(drop=True)
top_N = 10
df_valid_result = df_valid_result.groupby('did').head(top_N).reset_index(drop=True)
df_temp, final_score = cal_final_score (df_valid_result[['did', 'vid']], 
                               df_valid_label.select(['did', 'vid']).to_pandas(), 
                               weight=0.9)
print ("验证集得分为: ", final_score)

fold_importance_df = pd.DataFrame()
fold_importance_df["Feature"] = feature_name
fold_importance_df["importance"] = lgb_model.feature_importance()
fold_importance_df["importance_gain"] = lgb_model.feature_importance(importance_type='gain')
fold_importance_df = fold_importance_df.sort_values(by='importance', ascending=False)
fold_importance_df.to_csv("fold_importance_df.csv", index=None) 
fold_importance_df

cpu_counts:  16
Feature length = 37
Feature:  ['vid', 'count_vid_pair_sim_did', 'sum_vid_pair_sim_play_time_right', 'cid', 'is_intact', 'classify_id', 'serialno', 'duration', 'online_time_gap', 'max_did_history_click_play_time', 'mean_did_history_click_play_time', 'std_did_history_click_play_time', 'min_did_history_click_play_time', 'count_did_history_click_vid', 'max_vid_history_click_play_time', 'mean_vid_history_click_play_time', 'std_vid_history_click_play_time', 'min_vid_history_click_play_time', 'count_vid_history_click_did', 'max_cid_history_click_play_time', 'mean_cid_history_click_play_time', 'std_cid_history_click_play_time', 'min_cid_history_click_play_time', 'count_cid_history_click_did', 'count_did_vid_history_click_play_time', 'max_did_cid_history_click_play_time', 'mean_did_cid_history_click_play_time', 'std_did_cid_history_click_play_time', 'min_did_cid_history_click_play_time', 'count_did_cid_history_click_play_time', 'count_did_history_show_vid', 'count_vid_history_sh

,Feature,importance,importance_gain
30,count_did_history_show_vid,2977,88419.634127
2,sum_vid_pair_sim_play_time_right,2237,39538.516698
1,count_vid_pair_sim_did,2013,72617.480158
11,std_did_history_click_play_time,1917,19465.257035
10,mean_did_history_click_play_time,1634,14825.410216
9,max_did_history_click_play_time,1570,17930.420955
18,count_vid_history_click_did,1459,232261.612790
35,vid_ctr,1459,39811.779829
0,vid,1443,39049.985978
12,min_did_history_click_play_time,1430,16261.650121


In [14]:
%%time

num = os.cpu_count()
print ("cpu_counts: ", num)

if 'predict_prob' in df_valid_sample.columns.values :
    del df_valid_sample['predict_prob']
df_total_sample = pd.concat([df_train_sample, df_valid_sample]).reset_index(drop=True)
train_label = df_total_sample['label'].values

#df_train_group = df_total_sample.groupby("did")["did"].count().to_numpy()


feature_name = lgb_model.feature_name()
print(f"Feature length = {len(feature_name)}")
print ("Feature: ", feature_name)

print("Model Trainning.")

trn_data = lgb.Dataset(df_total_sample[feature_name], 
                       label=train_label,)
                       #group=df_train_group)#, categorical_feature=cat_cols)

#print ("train_data : ", train_data.info())
#print ("test_data : ", train_data.info())
lgb_model = lgb.train(lgb_params,
                      trn_data,
                      int(lgb_model.best_iteration * 1.2),)

df_test_sample['predict_prob'] = lgb_model.predict(df_test_sample[feature_name], 
                                                    num_iteration=lgb_model.best_iteration)

#评估线下验证集得分
df_test_result = df_test_sample[['did', 'vid', 'predict_prob']].sort_values(by=['did', 'predict_prob'], ascending=False).reset_index(drop=True)
top_N = 10
df_test_result = df_test_result.groupby('did').head(top_N).reset_index(drop=True)
#保存答案
df_test_result[['did', 'vid']].to_csv('mgtv_rec_baseline_v1.csv', index=None)

import time
path = "baseline_online_model_v1/" + str(int(time.time()))
print(time.time())#获当前时间的时间戳
print(path)#获取本地时间
os.makedirs(path)
lgb_model.save_model(path + '/model.txt')


fold_importance_df = pd.DataFrame()
fold_importance_df["Feature"] = feature_name
fold_importance_df["importance"] = lgb_model.feature_importance()
fold_importance_df["importance_gain"] = lgb_model.feature_importance(importance_type='gain')
fold_importance_df = fold_importance_df.sort_values(by='importance', ascending=False)
fold_importance_df.to_csv("fold_importance_df.csv", index=None) 
fold_importance_df

cpu_counts:  16
Feature length = 37
Feature:  ['vid', 'count_vid_pair_sim_did', 'sum_vid_pair_sim_play_time_right', 'cid', 'is_intact', 'classify_id', 'serialno', 'duration', 'online_time_gap', 'max_did_history_click_play_time', 'mean_did_history_click_play_time', 'std_did_history_click_play_time', 'min_did_history_click_play_time', 'count_did_history_click_vid', 'max_vid_history_click_play_time', 'mean_vid_history_click_play_time', 'std_vid_history_click_play_time', 'min_vid_history_click_play_time', 'count_vid_history_click_did', 'max_cid_history_click_play_time', 'mean_cid_history_click_play_time', 'std_cid_history_click_play_time', 'min_cid_history_click_play_time', 'count_cid_history_click_did', 'count_did_vid_history_click_play_time', 'max_did_cid_history_click_play_time', 'mean_did_cid_history_click_play_time', 'std_did_cid_history_click_play_time', 'min_did_cid_history_click_play_time', 'count_did_cid_history_click_play_time', 'count_did_history_show_vid', 'count_vid_history_sh

,Feature,importance,importance_gain
30,count_did_history_show_vid,3339,179109.192301
2,sum_vid_pair_sim_play_time_right,2529,64017.027459
1,count_vid_pair_sim_did,2268,104463.647107
11,std_did_history_click_play_time,2061,26236.395145
35,vid_ctr,1881,120859.373793
18,count_vid_history_click_did,1801,413546.428283
7,duration,1757,79131.512085
9,max_did_history_click_play_time,1687,24613.523729
0,vid,1671,61437.366893
10,mean_did_history_click_play_time,1616,18300.315679
